In [24]:
import numpy as np
import pandas as pd

In [25]:
categories = pd.read_csv('../../data_q2/q2-ucsd-cat-map.csv')
consumer = pd.read_parquet('../../data_q2/q2-ucsd-consDF.pqt')
acct = pd.read_parquet('../../data_q2/q2-ucsd-acctDF.pqt')
transactions = pd.read_parquet('../../data_q2/q2-ucsd-trxnDF.pqt')

In [26]:
# categories = pd.read_csv('../../data_q2/q2-ucsd-cat-map.csv')
# consumer = pd.read_parquet('../../data_q2/q2-ucsd-consDF.pqt')
# acct = pd.read_parquet("../../data_q2/q2-ucsd-acctIDF.pqt")
# transactions = pd.read_parquet("../../data_q2/q2-ucsd-trxnDF.pqt")

In [27]:
consumer.head()

,prism_consumer_id,evaluation_date,credit_score,DQ_TARGET
0,0,2021-09-01,726.0,0.0
1,1,2021-07-01,626.0,0.0
2,2,2021-05-01,680.0,0.0
3,3,2021-03-01,734.0,0.0
4,4,2021-10-01,676.0,0.0


In [79]:
# consumer.groupby('DQ_TARGET').count() #10994 0.0 vs 1006 1.0


In [28]:
transactions.head()

,prism_consumer_id,prism_transaction_id,category,amount,credit_or_debit,posted_date
0,3023,0,4,0.05,CREDIT,2021-04-16
1,3023,1,12,481.56,CREDIT,2021-04-30
2,3023,2,4,0.05,CREDIT,2021-05-16
3,3023,3,4,0.07,CREDIT,2021-06-16
4,3023,4,4,0.06,CREDIT,2021-07-16


In [29]:
categories.head()

,category_id,category
0,0,SELF_TRANSFER
1,1,EXTERNAL_TRANSFER
2,2,DEPOSIT
3,3,PAYCHECK
4,4,MISCELLANEOUS


In [30]:
acct.head()

,prism_consumer_id,prism_account_id,account_type,balance_date,balance
0,3023,0,SAVINGS,2021-08-31,90.57
1,3023,1,CHECKING,2021-08-31,225.95
2,4416,2,SAVINGS,2022-03-31,15157.17
3,4416,3,CHECKING,2022-03-31,66.42
4,4227,4,CHECKING,2021-07-31,7042.90


In [31]:
len(acct['prism_consumer_id'].unique())

13009

In [32]:
# merge 
acct['account_type'].unique()

# most important account_types: ['CHECKING', 'SAVINGS', 'CREDIT CARD', 'LOAN]
# most_important_accounts = ['CHECKING', 'SAVINGS', 'CREDIT CARD', 'LOAN']
acctDF = acct.copy()
total_balance = acctDF.groupby('prism_consumer_id')['balance'].sum()
total_balance

pd.DataFrame(total_balance) #get total balance for each consumer

,balance
prism_consumer_id,
0,320.37
1,3302.42
10,824.24
100,2655.47
1000,95.25
...,...
9995,0.00
9996,252.67
9997,611.28


In [33]:
consumer_balance = consumer.merge(pd.DataFrame(total_balance), on = 'prism_consumer_id', how = 'outer')
consumer_balance['std_credit'] = (consumer_balance['credit_score'] - consumer_balance['credit_score'].mean()) / consumer_balance['credit_score'].std()
consumer_balance["std_balance"] = (
    consumer_balance["balance"] - consumer_balance["balance"].mean()
    ) / consumer_balance["balance"].std()
consumer_balance

,prism_consumer_id,evaluation_date,credit_score,DQ_TARGET,balance,std_credit,std_balance
0,0,2021-09-01,726.0,0.0,320.37,0.846851,-0.146222
1,1,2021-07-01,626.0,0.0,3302.42,-0.459894,-0.090027
2,10,2022-02-01,654.0,0.0,824.24,-0.094005,-0.136727
3,100,2021-12-01,750.0,0.0,2655.47,1.160470,-0.102219
4,1000,2021-03-01,756.0,0.0,95.25,1.238874,-0.150464
...,...,...,...,...,...,...,...
14995,9995,2023-03-02,578.0,NaN,0.00,-1.087132,-0.152259
14996,9996,2023-10-11,610.0,NaN,252.67,-0.668973,-0.147498
14997,9997,2023-05-25,675.0,NaN,611.28,0.180411,-0.140740
14998,9998,2023-11-02,534.0,NaN,-862.99,-1.662099,-0.168522


In [56]:
!pip install xgboost
# !pip uninstall xgboost -y
import xgboost


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [60]:
!pip install lightgbm
import lightgbm as lgb
print(lgb.__version__)


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
4.5.0


In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import UndefinedMetricWarning
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
import warnings

def run_classification(
    feature_column, target_column, dataset, test_size=0.2, random_state=42
):
    """
    Run Logistic Regression and Random Forest Classification on a dataset.

    Parameters:
        feature_column (list): List of columns that can be used as features.
        target_column (str): The name of the target column.
        dataset (pd.DataFrame): The dataset containing the features and target.
        test_size (float): Proportion of the dataset to include in the test split (default 0.3).
        random_state (int): Random seed for reproducibility (default 42).

    Returns:
        None: Prints the classification reports for both models.
    """
    warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

    # Drop NaN values and shuffle the dataset
    dataset = dataset.dropna()
    dataset = dataset.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Define features and target
    X = dataset[feature_column]
    y = dataset[target_column]

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # balance the dataset
    sm = SMOTE(random_state=42)
    X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

    # Logistic Regression
    log_model = LogisticRegression(class_weight = 'balanced')
    log_model.fit(X_train_bal, y_train_bal)
    log_y_pred = log_model.predict(X_test)
    print(f"Logistic Regression for {feature_column}")
    print(classification_report(y_test, log_y_pred))

    # Random Forest Classification
    rfc_model = RandomForestClassifier(random_state=random_state)
    rfc_model.fit(X_train_bal, y_train_bal)
    rfc_y_pred = rfc_model.predict(X_test)
    print(f"\nRandom Forest Classification for {feature_column}")
    print(classification_report(y_test, rfc_y_pred))

    # Light GBM 
    lgb_model = lgb.LGBMClassifier()
    lgb_model.fit(X_train_bal, y_train_bal)
    lgb_y_pred = lgb_model.predict(X_test)
    print(f"\nLGB Model Classification for {feature_column}")
    print(classification_report(y_test, lgb_y_pred))

    # SVC Model
    svc_model = SVC(probability=True, random_state=random_state)
    svc_model.fit(X_train_bal, y_train_bal)
    svc_y_pred = svc_model.predict(X_test)
    print(f"\nSVM Classification for {feature_column}")
    print(classification_report(y_test, svc_y_pred))

    # XGB Model
    xgb_model = xgb.XGBClassifier(
        scale_pos_weight=len(y_train) / y_train.sum()
        )
    xgb_model.fit(X_train_bal, y_train_bal)
    xgb_y_pred = xgb_model.predict(X_test)
    print("\nXGBoost:")
    print(classification_report(y_test, xgb_y_pred))

In [71]:
# Added ROC AUC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import UndefinedMetricWarning
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
import warnings

def run_classification2(
    feature_column, target_column, dataset, test_size=0.2, random_state=42
):
    """
    Run Logistic Regression, Random Forest, LightGBM, SVM, and XGBoost Classification on a dataset.
    Includes ROC-AUC evaluation for all models.

    Parameters:
        feature_column (list): List of columns that can be used as features.
        target_column (str): The name of the target column.
        dataset (pd.DataFrame): The dataset containing the features and target.
        test_size (float): Proportion of the dataset to include in the test split (default 0.2).
        random_state (int): Random seed for reproducibility (default 42).

    Returns:
        None: Prints the classification reports and ROC-AUC scores for all models.
    """
    warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

    # Drop NaN values and shuffle the dataset
    dataset = dataset.dropna()
    dataset = dataset.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Define features and target
    X = dataset[feature_column]
    y = dataset[target_column]

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Balance the dataset using SMOTE
    sm = SMOTE(random_state=42)
    X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

    # Logistic Regression
    log_model = LogisticRegression(class_weight='balanced')
    log_model.fit(X_train_bal, y_train_bal)
    log_y_pred = log_model.predict(X_test)
    log_y_proba = log_model.predict_proba(X_test)[:, 1]
    print(f"Logistic Regression for {feature_column}")
    print(classification_report(y_test, log_y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, log_y_proba):.3f}")

    # Random Forest Classification
    rfc_model = RandomForestClassifier(random_state=random_state)
    rfc_model.fit(X_train_bal, y_train_bal)
    rfc_y_pred = rfc_model.predict(X_test)
    rfc_y_proba = rfc_model.predict_proba(X_test)[:, 1]
    print(f"\nRandom Forest Classification for {feature_column}")
    print(classification_report(y_test, rfc_y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, rfc_y_proba):.3f}")

    # LightGBM
    lgb_model = lgb.LGBMClassifier()
    lgb_model.fit(X_train_bal, y_train_bal)
    lgb_y_pred = lgb_model.predict(X_test)
    lgb_y_proba = lgb_model.predict_proba(X_test)[:, 1]
    print(f"\nLightGBM Classification for {feature_column}")
    print(classification_report(y_test, lgb_y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, lgb_y_proba):.3f}")

    # SVM Classification
    svc_model = SVC(probability=True, random_state=random_state)
    svc_model.fit(X_train_bal, y_train_bal)
    svc_y_pred = svc_model.predict(X_test)
    svc_y_proba = svc_model.predict_proba(X_test)[:, 1]
    print(f"\nSVM Classification for {feature_column}")
    print(classification_report(y_test, svc_y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, svc_y_proba):.3f}")

    # XGBoost Classification
    xgb_model = xgb.XGBClassifier(scale_pos_weight=len(y_train) / y_train.sum())
    xgb_model.fit(X_train_bal, y_train_bal)
    xgb_y_pred = xgb_model.predict(X_test)
    xgb_y_proba = xgb_model.predict_proba(X_test)[:, 1]
    print(f"\nXGBoost Classification for {feature_column}")
    print(classification_report(y_test, xgb_y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, xgb_y_proba):.3f}")


In [72]:
run_classification(["std_credit"], "DQ_TARGET", consumer_balance)

Logistic Regression for ['std_credit']
              precision    recall  f1-score   support

         0.0       0.95      0.74      0.83      1897
         1.0       0.19      0.63      0.29       185

    accuracy                           0.73      2082
   macro avg       0.57      0.68      0.56      2082
weighted avg       0.89      0.73      0.78      2082


Random Forest Classification for ['std_credit']
              precision    recall  f1-score   support

         0.0       0.93      0.81      0.87      1897
         1.0       0.16      0.38      0.23       185

    accuracy                           0.77      2082
   macro avg       0.55      0.60      0.55      2082
weighted avg       0.86      0.77      0.81      2082

[LightGBM] [Info] Number of positive: 7596, number of negative: 7596
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000108 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bin

In [73]:
# Example usage:
run_classification2(["std_credit"], "DQ_TARGET", consumer_balance)

Logistic Regression for ['std_credit']
              precision    recall  f1-score   support

         0.0       0.95      0.74      0.83      1897
         1.0       0.19      0.63      0.29       185

    accuracy                           0.73      2082
   macro avg       0.57      0.68      0.56      2082
weighted avg       0.89      0.73      0.78      2082

ROC-AUC Score: 0.750

Random Forest Classification for ['std_credit']
              precision    recall  f1-score   support

         0.0       0.93      0.81      0.87      1897
         1.0       0.16      0.38      0.23       185

    accuracy                           0.77      2082
   macro avg       0.55      0.60      0.55      2082
weighted avg       0.86      0.77      0.81      2082

ROC-AUC Score: 0.662
[LightGBM] [Info] Number of positive: 7596, number of negative: 7596
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000122 seconds.
You can set `force_col_wise=true` to remove

In [74]:
# Example usage:  #features, target, dataframe
run_classification(["std_balance"], "DQ_TARGET", consumer_balance)

Logistic Regression for ['std_balance']
              precision    recall  f1-score   support

         0.0       0.96      0.36      0.52      1897
         1.0       0.11      0.85      0.20       185

    accuracy                           0.40      2082
   macro avg       0.54      0.61      0.36      2082
weighted avg       0.89      0.40      0.49      2082


Random Forest Classification for ['std_balance']
              precision    recall  f1-score   support

         0.0       0.92      0.68      0.79      1897
         1.0       0.11      0.41      0.18       185

    accuracy                           0.66      2082
   macro avg       0.52      0.55      0.48      2082
weighted avg       0.85      0.66      0.73      2082

[LightGBM] [Info] Number of positive: 7596, number of negative: 7596
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000153 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total B

In [75]:
run_classification2(["std_balance"], "DQ_TARGET", consumer_balance)

Logistic Regression for ['std_balance']
              precision    recall  f1-score   support

         0.0       0.96      0.36      0.52      1897
         1.0       0.11      0.85      0.20       185

    accuracy                           0.40      2082
   macro avg       0.54      0.61      0.36      2082
weighted avg       0.89      0.40      0.49      2082

ROC-AUC Score: 0.701

Random Forest Classification for ['std_balance']
              precision    recall  f1-score   support

         0.0       0.92      0.68      0.79      1897
         1.0       0.11      0.41      0.18       185

    accuracy                           0.66      2082
   macro avg       0.52      0.55      0.48      2082
weighted avg       0.85      0.66      0.73      2082

ROC-AUC Score: 0.589
[LightGBM] [Info] Number of positive: 7596, number of negative: 7596
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000124 seconds.
You can set `force_col_wise=true` to remo

In [80]:
#NEW FEATURES: 

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE

# Load datasets
transactions = pd.read_parquet('../../data_q2/q2-ucsd-trxnDF.pqt')
consumer = pd.read_parquet('../../data_q2/q2-ucsd-consDF.pqt')

# Ensure date is in datetime format
transactions['posted_date'] = pd.to_datetime(transactions['posted_date'])

# Separate credit and debit transactions
credit_transactions = transactions[transactions['credit_or_debit'] == 'CREDIT']
debit_transactions = transactions[transactions['credit_or_debit'] == 'DEBIT']

# Aggregate total credit and debit amounts per consumer
credit_features = credit_transactions.groupby('prism_consumer_id').agg(
    total_credit=('amount', 'sum')
).reset_index()

debit_features = debit_transactions.groupby('prism_consumer_id').agg(
    total_debit=('amount', 'sum')
).reset_index()

# Merge credit and debit features
credit_debit_features = credit_features.merge(debit_features, on='prism_consumer_id', how='outer').fillna(0)

# Merge with consumer data
consumer_features = consumer.merge(credit_debit_features, on='prism_consumer_id', how='left').fillna(0)

# Prepare dataset for modeling
selected_features = ['total_credit', 'total_debit']
X = consumer_features[selected_features]
y = consumer_features['DQ_TARGET']

# Drop NA values and split the data
X = X.dropna()
y = y[X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Balance the dataset
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

# Train Logistic Regression
model = LogisticRegression(class_weight='balanced')
model.fit(X_train_bal, y_train_bal)

# Predictions and Evaluation
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred))

roc_auc = roc_auc_score(y_test, y_proba)
print(f"ROC-AUC Score: {roc_auc:.3f}")


Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.37      0.53      2798
         1.0       0.08      0.80      0.15       202

    accuracy                           0.40      3000
   macro avg       0.52      0.58      0.34      3000
weighted avg       0.90      0.40      0.51      3000

ROC-AUC Score: 0.607


In [ ]:
# run_classification2(["std_balance"], "DQ_TARGET", consumer_balance)

In [82]:
transactions

,prism_consumer_id,prism_transaction_id,category,amount,credit_or_debit,posted_date
0,3023,0,4,0.05,CREDIT,2021-04-16
1,3023,1,12,481.56,CREDIT,2021-04-30
2,3023,2,4,0.05,CREDIT,2021-05-16
3,3023,3,4,0.07,CREDIT,2021-06-16
4,3023,4,4,0.06,CREDIT,2021-07-16
...,...,...,...,...,...,...
6407316,10533,6405304,31,4.96,DEBIT,2022-03-11
6407317,10533,6405305,12,63.48,DEBIT,2022-03-30
6407318,10533,6405306,12,53.99,DEBIT,2022-03-30
6407319,10533,6405307,12,175.98,DEBIT,2022-03-31


In [81]:
credit_transactions = transactions[transactions['credit_or_debit'] == 'CREDIT']
debit_transactions = transactions[transactions['credit_or_debit'] == 'DEBIT']

credit_features = credit_transactions.groupby('prism_consumer_id').agg(
    total_credit=('amount', 'sum')
).reset_index()

debit_features = debit_transactions.groupby('prism_consumer_id').agg(
    total_debit=('amount', 'sum')
).reset_index()

# Merge and prepare data
credit_debit_features = credit_features.merge(debit_features, on='prism_consumer_id', how='outer').fillna(0)
consumer_features = consumer.merge(credit_debit_features, on='prism_consumer_id', how='left').fillna(0)

# Run classification on credit and debit features
run_classification2(['total_credit', 'total_debit'], 'DQ_TARGET', consumer_features)

Logistic Regression for ['total_credit', 'total_debit']
              precision    recall  f1-score   support

         0.0       0.96      0.36      0.52      2784
         1.0       0.09      0.79      0.16       216

    accuracy                           0.39      3000
   macro avg       0.52      0.57      0.34      3000
weighted avg       0.89      0.39      0.49      3000

ROC-AUC Score: 0.604

Random Forest Classification for ['total_credit', 'total_debit']
              precision    recall  f1-score   support

         0.0       0.94      0.76      0.84      2784
         1.0       0.11      0.39      0.17       216

    accuracy                           0.73      3000
   macro avg       0.53      0.57      0.51      3000
weighted avg       0.88      0.73      0.79      3000

ROC-AUC Score: 0.600
[LightGBM] [Info] Number of positive: 11210, number of negative: 11210
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000850 seconds.
You can

In [83]:
credit_features

,prism_consumer_id,total_credit
0,0,14386.82
1,1,24903.80
2,10,20576.56
3,100,35236.84
4,1000,78353.07
...,...,...
14471,9995,16925.84
14472,9996,1200.03
14473,9997,17206.11
14474,9998,14566.37


In [ ]:
consumer_balance['DQ_TARGET'].value_counts()

DQ_TARGET
0.0    10994
1.0     1006
Name: count, dtype: int64

In [ ]:
consumer_balance.isna().sum()

prism_consumer_id       0
evaluation_date         0
credit_score            0
DQ_TARGET            3000
balance              1991
std_credit              0
std_balance          1991
dtype: int64

In [ ]:
new_data = consumer_balance.dropna()
new_data['DQ_TARGET'].value_counts()

DQ_TARGET
0.0    9493
1.0     915
Name: count, dtype: int64